In [194]:
# imports
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')   
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression   
from sklearn.metrics import mean_squared_error, r2_score    
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso   



# Energy Anomaly & Power Theft Detection - Data Analysis Notebook

## Overview
This notebook loads, cleans, integrates, and analyzes multiple electricity consumption and operational datasets for anomaly detection.

## Notebook Structure
1. **Imports & Setup** - Required libraries
2. **Section A: Load All Datasets** - Power, weather, KPLC schedules, and auxiliary files
3. **Section B: Data Cleaning & Preprocessing** - Handle missing values, normalize types, engineer features
4. **Section C: Exploratory Data Analysis** - Summary statistics, data quality checks
5. **Section D: Data Integration & Merging** - Combine datasets on common keys (date, meter_id)
6. **Section E: Feature Engineering & Anomaly Detection** - Build features for modeling

---

## Section A: Load All Datasets

In [195]:
# 1. Load: power_multi_household_daily.csv (primary power dataset)
power_path = "power_multi_household_daily.csv"
df_power_daily = pd.read_csv(power_path, low_memory=False)
print(f"✓ Loaded {power_path}: {df_power_daily.shape}")

✓ Loaded power_multi_household_daily.csv: (14420, 9)


In [196]:
# 2. Load: nairobi_weather_2007_2008.csv (weather context)
weather_path = "nairobi_weather_2007_2008.csv"
df_weather = pd.read_csv(weather_path, low_memory=False)
print(f"✓ Loaded {weather_path}: {df_weather.shape}")

✓ Loaded nairobi_weather_2007_2008.csv: (731, 5)


In [197]:
# 3. Load: kplc_daily_schedule.csv (scheduled outages)
kplc_path = "kplc_daily_schedule.csv"
df_kplc = pd.read_csv(kplc_path, low_memory=False)
print(f"✓ Loaded {kplc_path}: {df_kplc.shape}")

✓ Loaded kplc_daily_schedule.csv: (5, 4)


In [237]:
# 4. Load: kplc_planned_outages.csv (maintenance records)
planned_outages_path = "kplc_planned_outages.csv"
df_planned_outages = pd.read_csv(planned_outages_path, low_memory=False)
print(f"✓ Loaded {planned_outages_path}: {df_planned_outages.shape}")

✓ Loaded kplc_planned_outages.csv: (13, 5)


In [199]:
# 5. Load: df.csv (auxiliary data)
df_aux = pd.read_csv("df.csv", low_memory=False)
print(f"✓ Loaded df.csv: {df_aux.shape}")

✓ Loaded df.csv: (560655, 13)


In [200]:
# 6. Load: lead1.0-small.csv (lead dataset)
lead_path = "lead1.0-small.csv"
df_lead = pd.read_csv(lead_path, low_memory=False)
print(f"✓ Loaded {lead_path}: {df_lead.shape}")

print("\n" + "="*80)
print("ALL DATASETS LOADED SUMMARY")
print("="*80)
datasets = {
    'df_power_daily': df_power_daily,
    'df_weather': df_weather,
    'df_kplc': df_kplc,
    'df_planned_outages': df_planned_outages,
    'df_aux': df_aux,
    'df_lead': df_lead
}
for name, df in datasets.items():
    print(f"{name:25} | Shape: {str(df.shape):12} | Columns: {list(df.columns)[:3]}...")
print("="*80)

✓ Loaded lead1.0-small.csv: (1749494, 4)

ALL DATASETS LOADED SUMMARY
df_power_daily            | Shape: (14420, 9)   | Columns: ['meter_id', 'date', 'daily_mean_power']...
df_weather                | Shape: (731, 5)     | Columns: ['date', 'tmax', 'tmin']...
df_kplc                   | Shape: (5, 4)       | Columns: ['date', 'scheduled_outage_today', 'n_scheduled_events']...
df_planned_outages        | Shape: (13, 5)      | Columns: ['area', 'date', 'start_time']...
df_aux                    | Shape: (560655, 13) | Columns: ['0', 'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)']...
df_lead                   | Shape: (1749494, 4) | Columns: ['building_id', 'timestamp', 'meter_reading']...


In [201]:
# Loading dataset: power_multi_household_daily.csv
power_path = "power_multi_household_daily.csv"

df_power_daily = pd.read_csv(power_path, low_memory=False)
df_power_daily

,meter_id,date,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean
0,MTR_001,2006-12-16,2.617495,0.882098,0.222336,6.660460,236.243763,2.922896,13.082828
1,MTR_001,2006-12-17,2.018833,1.032528,0.097898,6.024453,240.087028,4.051467,9.999028
2,MTR_001,2006-12-18,1.311082,0.862724,0.065486,5.258530,241.231694,3.719576,6.421667
3,MTR_001,2006-12-19,0.992268,1.063872,0.020072,6.750390,241.999313,3.069492,4.926389
4,MTR_001,2006-12-20,1.325185,1.134982,0.026983,5.161571,242.308063,3.345704,6.467361
...,...,...,...,...,...,...,...,...,...
14415,MTR_010,2010-11-22,1.624360,1.346636,0.146076,6.865426,240.197118,3.916832,6.014167
14416,MTR_010,2010-11-23,1.254788,1.004583,0.094465,5.337459,240.575528,3.833612,4.674444
14417,MTR_010,2010-11-24,1.431164,1.020773,0.133426,6.534909,240.030965,3.747974,5.249583
14418,MTR_010,2010-11-25,1.139526,0.955009,0.156308,5.809185,241.536257,2.714365,4.169444


### B.1 Clean & Parse Power Dataset

In [202]:
# parse date and set index if present
if 'date' in df_power_daily.columns:
    df_power_daily['date'] = pd.to_datetime(df_power_daily['date'], errors='coerce')
    df_power_daily.set_index('date', inplace=True)


In [234]:
# basic preview and diagnostics
print(f"Loaded '{power_path}' — shape: {df_power_daily.shape}")
print('\nColumn dtypes:')
print(df_power_daily.dtypes)



Loaded 'power_multi_household_daily.csv' — shape: (14330, 8)

Column dtypes:
meter_id             object
daily_mean_power    float64
daily_std_power     float64
daily_min_power     float64
daily_max_power     float64
voltage_mean        float64
voltage_std         float64
intensity_mean      float64
dtype: object


In [204]:
# checking missing values
print('\nMissing values per column:')
print(df_power_daily.isnull().sum())


Missing values per column:
meter_id             0
daily_mean_power    90
daily_std_power     90
daily_min_power     90
daily_max_power     90
voltage_mean        90
voltage_std         90
intensity_mean      90
dtype: int64


In [205]:
# basic dataset-specific checks
if 'meter_id' in df_power_daily.columns:
    print('\nUnique meters:', df_power_daily['meter_id'].nunique())
for col in ['daily_consumption','consumption','Global_active_power','energy_kwh','total_kwh']:
    if col in df_power_daily.columns:
        print(f"\nSummary for '{col}':")
        display(df_power_daily[col].describe())


Unique meters: 10


In [206]:
# Loading dataset: kplc_daily_schedule.csv 
# file path
kplc_path = "kplc_daily_schedule.csv"

# load dataset
df_kplc = pd.read_csv(kplc_path, low_memory=False)
df_kplc


,date,scheduled_outage_today,n_scheduled_events,total_scheduled_minutes
0,2026-01-18,1,3,1380.0
1,2026-01-20,1,4,1950.0
2,2026-01-21,1,2,960.0
3,2026-01-22,1,3,1380.0
4,2026-01-23,1,1,450.0


### B.2 Clean & Parse KPLC & Weather Datasets

In [207]:
# loading dataset: nairobi_weather_2007_2008.csv
# file path
weather_path = "nairobi_weather_2007_2008.csv"

# load dataset
df_weather = pd.read_csv(weather_path, low_memory=False)
df_weather.head()


,date,tmax,tmin,prcp,wspd_max
0,2007-01-01,24.7,13.1,1.0,17.6
1,2007-01-02,23.2,13.7,0.1,17.4
2,2007-01-03,22.6,14.8,0.9,20.0
3,2007-01-04,21.5,14.9,2.3,21.4
4,2007-01-05,23.3,12.9,0.0,18.5


In [208]:
# parse any `date` column to datetime (safe conversion)
for df, name in [(df_kplc, 'df_kplc'), (df_weather, 'df_weather')]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [209]:
# quick previews and summaries
print(f"Loaded '{kplc_path}' — shape: {df_kplc.shape}")
print('\nColumn dtypes:')
print(df_kplc.dtypes)
print('\nMissing values per column:')
print(df_kplc.isnull().sum())
if 'date' in df_kplc.columns:
    print(f"{kplc_path} date range: {df_kplc['date'].min()} — {df_kplc['date'].max()}")

print('\n' + '-'*60 + '\n')

print(f"Loaded '{weather_path}' — shape: {df_weather.shape}")
print('\nColumn dtypes:')
print(df_weather.dtypes)
print('\nMissing values per column:')
print(df_weather.isnull().sum())
if 'date' in df_weather.columns:
    print(f"{weather_path} date range: {df_weather['date'].min()} — {df_weather['date'].max()}")

Loaded 'kplc_daily_schedule.csv' — shape: (5, 4)

Column dtypes:
date                       datetime64[ns]
scheduled_outage_today              int64
n_scheduled_events                  int64
total_scheduled_minutes           float64
dtype: object

Missing values per column:
date                       0
scheduled_outage_today     0
n_scheduled_events         0
total_scheduled_minutes    0
dtype: int64
kplc_daily_schedule.csv date range: 2026-01-18 00:00:00 — 2026-01-23 00:00:00

------------------------------------------------------------

Loaded 'nairobi_weather_2007_2008.csv' — shape: (731, 5)

Column dtypes:
date        datetime64[ns]
tmax               float64
tmin               float64
prcp               float64
wspd_max           float64
dtype: object

Missing values per column:
date        0
tmax        0
tmin        0
prcp        0
wspd_max    0
dtype: int64
nairobi_weather_2007_2008.csv date range: 2007-01-01 00:00:00 — 2008-12-31 00:00:00


In [210]:
# make copies
df_kplc_clean = df_kplc.copy()
df_weather_clean = df_weather.copy()

In [211]:
# df_kplc cleaning
print('df_kplc: original shape ->', df_kplc_clean.shape)


df_kplc: original shape -> (5, 4)


In [212]:
# drop exact duplicate rows
dup_k = df_kplc_clean.duplicated().sum()
if dup_k:
    df_kplc_clean = df_kplc_clean.drop_duplicates()
    print(f'dropped {dup_k} duplicate rows from df_kplc')

In [213]:
# ensure numeric types
num_cols_k = ['scheduled_outage_today', 'n_scheduled_events', 'total_scheduled_minutes']
for c in num_cols_k:
    if c in df_kplc_clean.columns:
        df_kplc_clean[c] = pd.to_numeric(df_kplc_clean[c], errors='coerce')

In [214]:
# create date features
if 'date' in df_kplc_clean.columns and pd.api.types.is_datetime64_any_dtype(df_kplc_clean['date']):
    df_kplc_clean = df_kplc_clean.sort_values('date')
    df_kplc_clean['year'] = df_kplc_clean['date'].dt.year
    df_kplc_clean['month'] = df_kplc_clean['date'].dt.month
    df_kplc_clean['day'] = df_kplc_clean['date'].dt.day
    df_kplc_clean['weekday'] = df_kplc_clean['date'].dt.weekday
    df_kplc_clean['is_weekend'] = df_kplc_clean['weekday'] >= 5
    df_kplc_clean.set_index('date', inplace=True)


In [215]:
# outlier flag for total minutes (IQR)
if 'total_scheduled_minutes' in df_kplc_clean.columns:
    q1 = df_kplc_clean['total_scheduled_minutes'].quantile(0.25)
    q3 = df_kplc_clean['total_scheduled_minutes'].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    df_kplc_clean['total_minutes_outlier'] = ((df_kplc_clean['total_scheduled_minutes'] < lower) | (df_kplc_clean['total_scheduled_minutes'] > upper))

In [216]:
# simple imputation placeholder (only apply if missing exists)
if df_kplc_clean.isnull().sum().sum() > 0:
    df_kplc_clean['total_scheduled_minutes'].fillna(df_kplc_clean['total_scheduled_minutes'].median(), inplace=True)

print('df_kplc_clean shape ->', df_kplc_clean.shape)
print('-'*60)


df_kplc_clean shape -> (5, 9)
------------------------------------------------------------


In [217]:
# df_weather cleaning 
df_weather_clean.shape


(731, 5)

In [218]:
# drop exact duplicate rows
dup_w = df_weather_clean.duplicated().sum()
if dup_w:
    df_weather_clean = df_weather_clean.drop_duplicates()
    print(f'dropped {dup_w} duplicate rows from df_weather')

In [219]:
# ensure numeric types for weather columns
for c in ['tmax', 'tmin', 'prcp', 'wspd_max']:
    if c in df_weather_clean.columns:
        df_weather_clean[c] = pd.to_numeric(df_weather_clean[c], errors='coerce')

In [220]:
# date features and index
if 'date' in df_weather_clean.columns and pd.api.types.is_datetime64_any_dtype(df_weather_clean['date']):
    df_weather_clean = df_weather_clean.sort_values('date')
    df_weather_clean['temp_range'] = df_weather_clean['tmax'] - df_weather_clean['tmin']
    df_weather_clean['precip_flag'] = df_weather_clean['prcp'] > 0
    df_weather_clean['year'] = df_weather_clean['date'].dt.year
    df_weather_clean['month'] = df_weather_clean['date'].dt.month
    df_weather_clean.set_index('date', inplace=True)

In [221]:
# forward/backward fill or interpolation if missing
if df_weather_clean.isnull().sum().sum() > 0:
    df_weather_clean = df_weather_clean.sort_index().interpolate(method='time').ffill().bfill()

print('df_weather_clean shape ->', df_weather_clean.shape)



df_weather_clean shape -> (731, 8)


In [222]:
# Attempting merge on date 
df_merged = pd.merge(df_kplc_clean.reset_index(), df_weather_clean.reset_index(), on='date', how='left', suffixes=('_kplc','_weather'))
print('Merged shape:', df_merged.shape)

Merged shape: (5, 18)


### D.1 Merge KPLC + Weather (Date-based)

In [223]:
# counting how many rows have matching weather
matched = df_merged['tmax'].notna().sum() if 'tmax' in df_merged.columns else 0
print('Rows with matching weather data:', matched)

Rows with matching weather data: 0


In [224]:
# checking date-range overlap
k_min, k_max = df_kplc_clean.index.min(), df_kplc_clean.index.max()
w_min, w_max = df_weather_clean.index.min(), df_weather_clean.index.max()
print(f"kplc date range: {k_min} — {k_max}")
print(f"weather date range: {w_min} — {w_max}")

kplc date range: 2026-01-18 00:00:00 — 2026-01-23 00:00:00
weather date range: 2007-01-01 00:00:00 — 2008-12-31 00:00:00


In [225]:
# quick preview of cleaned / merged data
print('\nSample (df_kplc_clean):')
print('\nSample (df_weather_clean):')
print('\nSample (df_merged):')
display(df_merged.head())


Sample (df_kplc_clean):

Sample (df_weather_clean):

Sample (df_merged):


,date,scheduled_outage_today,n_scheduled_events,total_scheduled_minutes,year_kplc,month_kplc,day,weekday,is_weekend,total_minutes_outlier,tmax,tmin,prcp,wspd_max,temp_range,precip_flag,year_weather,month_weather
0,2026-01-18,1,3,1380.0,2026,1,18,6,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2026-01-20,1,4,1950.0,2026,1,20,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-21,1,2,960.0,2026,1,21,2,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-22,1,3,1380.0,2026,1,22,3,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-01-23,1,1,450.0,2026,1,23,4,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
# Info check for the 3 main datasets
print("=" * 80)
print("Dataset 1: df_power_daily")
print("=" * 80)
df_power_daily.info()

Dataset 1: df_power_daily
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14420 entries, 2006-12-16 to 2010-11-26
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   meter_id          14420 non-null  object 
 1   daily_mean_power  14330 non-null  float64
 2   daily_std_power   14330 non-null  float64
 3   daily_min_power   14330 non-null  float64
 4   daily_max_power   14330 non-null  float64
 5   voltage_mean      14330 non-null  float64
 6   voltage_std       14330 non-null  float64
 7   intensity_mean    14330 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1013.9+ KB


### C.1 Dataset Info & Summary Statistics

In [227]:
print("\n" + "=" * 80)
print("Dataset 2: df_kplc")
print("=" * 80)
df_kplc.info()


Dataset 2: df_kplc
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     5 non-null      datetime64[ns]
 1   scheduled_outage_today   5 non-null      int64         
 2   n_scheduled_events       5 non-null      int64         
 3   total_scheduled_minutes  5 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 292.0 bytes


In [228]:
print("\n" + "=" * 80)
print("Dataset 3: df_weather")
print("=" * 80)
df_weather.info()


Dataset 3: df_weather
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      731 non-null    datetime64[ns]
 1   tmax      731 non-null    float64       
 2   tmin      731 non-null    float64       
 3   prcp      731 non-null    float64       
 4   wspd_max  731 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 28.7 KB


In [229]:
# --- Merge: Power Multi-Household Daily + Nairobi Weather
print("Merging df_power_daily and df_weather on date...")

# Reset power index to make date a column for merging
power_reset = df_power_daily.reset_index()

# Ensure both date columns are datetime
power_reset['date'] = pd.to_datetime(power_reset['date'], errors='coerce')
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')

# Merge on date
df_power_weather = pd.merge(
    power_reset,
    df_weather,
    on='date',
    how='inner'  # inner join to keep only overlapping dates
)

print(f"\nMerged dataset 'df_power_weather':")
print(f"  Shape: {df_power_weather.shape}")
print(f"  Date range: {df_power_weather['date'].min()} → {df_power_weather['date'].max()}")

print(f"\nColumns ({df_power_weather.shape[1]}):")
print(df_power_weather.columns.tolist())

print(f"\nMissing values:")
print(df_power_weather.isnull().sum())

print(f"\nPreview:")
display(df_power_weather.head())

Merging df_power_daily and df_weather on date...

Merged dataset 'df_power_weather':
  Shape: (7310, 13)
  Date range: 2007-01-01 00:00:00 → 2008-12-31 00:00:00

Columns (13):
['date', 'meter_id', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max']

Missing values:
date                 0
meter_id             0
daily_mean_power    10
daily_std_power     10
daily_min_power     10
daily_max_power     10
voltage_mean        10
voltage_std         10
intensity_mean      10
tmax                 0
tmin                 0
prcp                 0
wspd_max             0
dtype: int64

Preview:


,date,meter_id,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean,tmax,tmin,prcp,wspd_max
0,2007-01-01,MTR_001,1.634586,0.782358,0.086498,3.066800,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
1,2007-01-01,MTR_002,2.388304,1.142670,0.202939,4.471271,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
2,2007-01-01,MTR_003,1.312100,0.688413,0.041358,2.655460,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
3,2007-01-01,MTR_004,2.122846,1.016589,0.214765,3.969411,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
4,2007-01-01,MTR_005,1.976886,0.947051,0.170827,3.748946,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6


### D.2 Merge Power + Weather (Date & Meter-based)

In [230]:
# Analysis: Missing values in df_power_daily (meter_id 14420 vs others 14330

# Identify rows with meter_id but missing power metrics
power_cols = ['daily_mean_power', 'daily_std_power', 'daily_min_power', 
              'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean']

### E.1 Data Quality: Missing Value Analysis

In [231]:
# Rows where ANY power column is null
incomplete_rows = df_power_daily[df_power_daily[power_cols].isnull().any(axis=1)]

print(f"\nRows with meter_id but missing power metrics: {len(incomplete_rows)}")
print(f"  Total rows: {len(df_power_daily)}")
print(f"  Complete rows: {len(df_power_daily) - len(incomplete_rows)}")
print(f"  Incomplete rows (missing metrics): {len(incomplete_rows)}")


Rows with meter_id but missing power metrics: 90
  Total rows: 14420
  Complete rows: 14330
  Incomplete rows (missing metrics): 90


In [232]:
#  Drop incomplete rows from df_power_daily
print("Dropping rows with missing power metrics...")

power_cols = ['daily_mean_power', 'daily_std_power', 'daily_min_power', 
              'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean']

# Before
shape_before = df_power_daily.shape
print(f"\nBefore: {shape_before}")

# Dropping rows where ANY power column is null
df_power_daily = df_power_daily.dropna(subset=power_cols, how='any')

# After
shape_after = df_power_daily.shape
print(f"After:  {shape_after}")
print(f"Rows removed: {shape_before[0] - shape_after[0]}")

# Verify no missing values in power columns
print(f"\nMissing values in power columns (post-drop):")
print(df_power_daily[power_cols].isnull().sum())

print(f"\ndf_power_daily is now clean and ready for analysis.")
print(f"Unique meters: {df_power_daily['meter_id'].nunique()}")
print(f"Date range: {df_power_daily.index.min()} → {df_power_daily.index.max()}")

Dropping rows with missing power metrics...

Before: (14420, 8)
After:  (14330, 8)
Rows removed: 90

Missing values in power columns (post-drop):
daily_mean_power    0
daily_std_power     0
daily_min_power     0
daily_max_power     0
voltage_mean        0
voltage_std         0
intensity_mean      0
dtype: int64

df_power_daily is now clean and ready for analysis.
Unique meters: 10
Date range: 2006-12-16 00:00:00 → 2010-11-26 00:00:00


### E.2 Data Cleaning: Remove Incomplete Rows

In [233]:
# --- Merge CLEANED: Power Multi-Household Daily + Nairobi Weather
print("Merging cleaned df_power_daily with df_weather...")

# Reset power index to make date a column for merging
power_reset = df_power_daily.reset_index()

# Ensure both date columns are datetime
power_reset['date'] = pd.to_datetime(power_reset['date'], errors='coerce')
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')

# Merge on date (inner join keeps only overlapping dates)
df_power_weather_clean = pd.merge(
    power_reset,
    df_weather,
    on='date',
    how='inner'
)

print(f"\n✓ Merged dataset 'df_power_weather_clean':")
print(f"  Shape: {df_power_weather_clean.shape}")
print(f"  Date range: {df_power_weather_clean['date'].min()} → {df_power_weather_clean['date'].max()}")
print(f"  Columns: {df_power_weather_clean.shape[1]}")
print(f"  Unique meters: {df_power_weather_clean['meter_id'].nunique()}")
print(f"  Missing values: {df_power_weather_clean.isnull().sum().sum()}")

print(f"\nColumn list:")
print(df_power_weather_clean.columns.tolist())

print(f"\nPreview ():")
display(df_power_weather_clean)

Merging cleaned df_power_daily with df_weather...

✓ Merged dataset 'df_power_weather_clean':
  Shape: (7300, 13)
  Date range: 2007-01-01 00:00:00 → 2008-12-31 00:00:00
  Columns: 13
  Unique meters: 10
  Missing values: 0

Column list:
['date', 'meter_id', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max']

Preview ():


,date,meter_id,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean,tmax,tmin,prcp,wspd_max
0,2007-01-01,MTR_001,1.634586,0.782358,0.086498,3.066800,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
1,2007-01-01,MTR_002,2.388304,1.142670,0.202939,4.471271,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
2,2007-01-01,MTR_003,1.312100,0.688413,0.041358,2.655460,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
3,2007-01-01,MTR_004,2.122846,1.016589,0.214765,3.969411,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
4,2007-01-01,MTR_005,1.976886,0.947051,0.170827,3.748946,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7295,2008-12-31,MTR_006,1.025914,0.845020,0.050078,4.493726,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7296,2008-12-31,MTR_007,1.431837,1.216527,0.053739,6.642979,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7297,2008-12-31,MTR_008,1.484169,1.228541,0.094065,6.565422,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7298,2008-12-31,MTR_009,1.516974,1.254294,0.106465,6.716099,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3


In [240]:
print("Creating comprehensive integrated dataset...")

# Start with cleaned power + weather merge
df_comprehensive = df_power_weather_clean.copy()

print(f"Starting dataset shape: {df_comprehensive.shape}")
print(f"Columns: {list(df_comprehensive.columns)}")

# Merge with KPLC features (left join to preserve all power records)
if len(df_kplc_clean) > 0:
    # Reset index if needed to make 'date' a column
    kplc_reset = df_kplc_clean.reset_index() if 'date' in df_kplc_clean.index.names else df_kplc_clean.copy()
    
    # Select only available columns from KPLC
    kplc_cols_available = [col for col in ['date', 'year', 'month', 'day', 'weekday', 'is_weekend', 'scheduled_outage_today', 'total_scheduled_minutes', 'total_minutes_outlier'] if col in kplc_reset.columns]
    
    df_comprehensive = pd.merge(
        df_comprehensive,
        kplc_reset[kplc_cols_available],
        on='date',
        how='left',
        suffixes=('_power', '_kplc')
    )
    # Fill KPLC missing values with 0 (no scheduled outage on that date)
    numeric_kplc_cols = [col for col in ['scheduled_outage_today', 'total_scheduled_minutes', 'total_minutes_outlier'] if col in df_comprehensive.columns]
    for col in numeric_kplc_cols:
        df_comprehensive[col].fillna(0, inplace=True)
    print(f"After adding KPLC features: {df_comprehensive.shape}")

# Merge with planned outages info (flag if date was in maintenance window)
if len(df_planned_outages) > 0:
    # Parse dates in planned outages
    df_planned_outages['date'] = pd.to_datetime(df_planned_outages['date'], errors='coerce')
    
    # Select available columns for merge
    planned_cols = [col for col in df_planned_outages.columns if col in ['date', 'area', 'end_time']]
    if 'date' in planned_cols:
        planned_outages_unique = df_planned_outages[planned_cols].drop_duplicates()
        
        df_comprehensive = pd.merge(
            df_comprehensive,
            planned_outages_unique,
            on='date',
            how='left'
        )


Creating comprehensive integrated dataset...
Starting dataset shape: (7300, 13)
Columns: ['date', 'meter_id', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max']
After adding KPLC features: (7300, 21)


In [239]:
        # Add indicator for planned outage presence
        if 'area' in df_comprehensive.columns:
            df_comprehensive['has_planned_outage'] = df_comprehensive['area'].notna().astype(int)
            df_comprehensive['area'].fillna('none', inplace=True)
        print(f"After adding planned outages: {df_comprehensive.shape}")

print("\nFinal Comprehensive Dataset Info:")
print(f"Shape: {df_comprehensive.shape}")
print(f"\nColumns ({len(df_comprehensive.columns)}):")
print(df_comprehensive.columns.tolist())
print(f"\nData types:\n{df_comprehensive.dtypes}")
print(f"\nMissing values:\n{df_comprehensive.isnull().sum()}")
print(f"\nFirst few rows:")
df_comprehensive.head()

After adding planned outages: (7300, 24)

Final Comprehensive Dataset Info:
Shape: (7300, 24)

Columns (24):
['date', 'meter_id', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max', 'year', 'month', 'day', 'weekday', 'is_weekend', 'scheduled_outage_today', 'total_scheduled_minutes', 'total_minutes_outlier', 'area', 'end_time', 'has_planned_outage']

Data types:
date                       datetime64[ns]
meter_id                           object
daily_mean_power                  float64
daily_std_power                   float64
daily_min_power                   float64
daily_max_power                   float64
voltage_mean                      float64
voltage_std                       float64
intensity_mean                    float64
tmax                              float64
tmin                              float64
prcp                              float64
wspd_max              

,date,meter_id,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean,tmax,...,month,day,weekday,is_weekend,scheduled_outage_today,total_scheduled_minutes,total_minutes_outlier,area,end_time,has_planned_outage
0,2007-01-01,MTR_001,1.634586,0.782358,0.086498,3.066800,240.128979,2.216851,7.916944,24.7,...,NaN,NaN,NaN,NaN,0.0,0.0,0,none,NaN,1
1,2007-01-01,MTR_002,2.388304,1.142670,0.202939,4.471271,240.128979,2.216851,7.916944,24.7,...,NaN,NaN,NaN,NaN,0.0,0.0,0,none,NaN,1
2,2007-01-01,MTR_003,1.312100,0.688413,0.041358,2.655460,240.128979,2.216851,7.916944,24.7,...,NaN,NaN,NaN,NaN,0.0,0.0,0,none,NaN,1
3,2007-01-01,MTR_004,2.122846,1.016589,0.214765,3.969411,240.128979,2.216851,7.916944,24.7,...,NaN,NaN,NaN,NaN,0.0,0.0,0,none,NaN,1
4,2007-01-01,MTR_005,1.976886,0.947051,0.170827,3.748946,240.128979,2.216851,7.916944,24.7,...,NaN,NaN,NaN,NaN,0.0,0.0,0,none,NaN,1


### F. Explore Auxiliary & Lead Datasets

In [241]:
print("=" * 80)
print("AUXILIARY DATASET (df_aux)")
print("=" * 80)
print(f"Shape: {df_aux.shape}")
print(f"\nColumns: {df_aux.columns.tolist()}")
print(f"\nData Types:\n{df_aux.dtypes}")
print(f"\nFirst few rows:")
df_aux.head()

AUXILIARY DATASET (df_aux)
Shape: (560655, 13)

Columns: ['0', 'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)', 'Heating:Electricity [kW](Hourly)', 'InteriorLights:Electricity [kW](Hourly)', 'InteriorEquipment:Electricity [kW](Hourly)', 'Gas:Facility [kW](Hourly)', 'Heating:Gas [kW](Hourly)', 'InteriorEquipment:Gas [kW](Hourly)', 'Water Heater:WaterSystems:Gas [kW](Hourly)', 'Class', 'theft']

Data Types:
0                                               int64
Electricity:Facility [kW](Hourly)             float64
Fans:Electricity [kW](Hourly)                 float64
Cooling:Electricity [kW](Hourly)              float64
Heating:Electricity [kW](Hourly)              float64
InteriorLights:Electricity [kW](Hourly)       float64
InteriorEquipment:Electricity [kW](Hourly)    float64
Gas:Facility [kW](Hourly)                     float64
Heating:Gas [kW](Hourly)                      float64
InteriorEquipment:Gas [kW](Hourly)            fl

,0,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Gas [kW](Hourly),InteriorEquipment:Gas [kW](Hourly),Water Heater:WaterSystems:Gas [kW](Hourly),Class,theft
0,0,22.035977,3.586221,0.0,0.0,4.589925,8.1892,136.585903,123.999076,3.33988,9.246947,FullServiceRestaurant,Normal
1,1,14.649757,0.000000,0.0,0.0,1.529975,7.4902,3.359880,0.000000,3.33988,0.020000,FullServiceRestaurant,Normal
2,2,14.669567,0.000000,0.0,0.0,1.529975,7.4902,3.359880,0.000000,3.33988,0.020000,FullServiceRestaurant,Normal
3,3,14.677808,0.000000,0.0,0.0,1.529975,7.4902,3.931932,0.000000,3.33988,0.592052,FullServiceRestaurant,Normal
4,4,14.824794,0.000000,0.0,0.0,1.529975,7.4902,3.359880,0.000000,3.33988,0.020000,FullServiceRestaurant,Normal


In [242]:
print("\n" + "=" * 80)
print("LEAD DATASET (df_lead)")
print("=" * 80)
print(f"Shape: {df_lead.shape}")
print(f"\nColumns: {df_lead.columns.tolist()}")
print(f"\nData Types:\n{df_lead.dtypes}")
print(f"\nFirst few rows:")
df_lead.head()
print(f"\nMissing values:\n{df_lead.isnull().sum()}")
print(f"Memory usage: {df_lead.memory_usage(deep=True).sum() / 1e6:.2f} MB")


LEAD DATASET (df_lead)
Shape: (1749494, 4)

Columns: ['building_id', 'timestamp', 'meter_reading', 'anomaly']

Data Types:
building_id        int64
timestamp         object
meter_reading    float64
anomaly            int64
dtype: object

First few rows:

Missing values:
building_id           0
timestamp             0
meter_reading    107653
anomaly               0
dtype: int64
Memory usage: 174.95 MB


In [243]:
print("\n" + "=" * 80)
print("AUXILIARY DATASET - DETAILED INFO")
print("=" * 80)
print(f"Shape: {df_aux.shape}")
print(f"\nColumns: {df_aux.columns.tolist()}")
print(f"\nData Types:\n{df_aux.dtypes}")
print(f"\nMissing values:\n{df_aux.isnull().sum()}")
print(f"\nMemory usage: {df_aux.memory_usage(deep=True).sum() / 1e6:.2f} MB")
print(f"\nBasic statistics:")
df_aux.describe()


AUXILIARY DATASET - DETAILED INFO
Shape: (560655, 13)

Columns: ['0', 'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)', 'Heating:Electricity [kW](Hourly)', 'InteriorLights:Electricity [kW](Hourly)', 'InteriorEquipment:Electricity [kW](Hourly)', 'Gas:Facility [kW](Hourly)', 'Heating:Gas [kW](Hourly)', 'InteriorEquipment:Gas [kW](Hourly)', 'Water Heater:WaterSystems:Gas [kW](Hourly)', 'Class', 'theft']

Data Types:
0                                               int64
Electricity:Facility [kW](Hourly)             float64
Fans:Electricity [kW](Hourly)                 float64
Cooling:Electricity [kW](Hourly)              float64
Heating:Electricity [kW](Hourly)              float64
InteriorLights:Electricity [kW](Hourly)       float64
InteriorEquipment:Electricity [kW](Hourly)    float64
Gas:Facility [kW](Hourly)                     float64
Heating:Gas [kW](Hourly)                      float64
InteriorEquipment:Gas [kW](Hourly)      

,0,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Gas [kW](Hourly),InteriorEquipment:Gas [kW](Hourly),Water Heater:WaterSystems:Gas [kW](Hourly)
count,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000,560655.000000
mean,280327.000000,161.775593,13.798083,43.770035,0.836993,32.988238,42.631944,77.310902,53.917106,8.163785,15.230010
std,161847.301923,287.323654,24.078277,117.101272,6.123290,65.177701,73.493664,178.680306,157.721787,15.954861,52.634413
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,140163.500000,15.843234,0.240958,0.000000,0.000000,3.005316,4.950048,1.103003,0.000000,0.000000,0.014510
50%,280327.000000,48.197262,3.557599,0.078813,0.000000,9.233307,15.988889,11.905720,0.083226,0.451244,1.103452
75%,420490.500000,155.448375,14.962478,16.838516,0.000000,34.382155,41.401920,66.165380,36.185902,7.575135,8.517566
max,560654.000000,1726.432223,240.015298,890.621907,277.996520,448.566544,448.566544,4491.695087,4480.732902,91.799800,783.877898
